In [1]:
# https://towardsdatascience.com/data-driven-growth-with-python-part-1-know-your-metrics-812781e66a5b
from __future__ import division
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb


tx_data = pd.read_csv('../data/OnlineRetail.csv', encoding= 'unicode_escape')

In [16]:
# tx_data.head()

# Our "North Star" metric: Revenue. Active Customer Count * order count * average revenue per order 

# Some basic engineering to clean the data.
# converting InvoiceDate from a string to a datetime. 
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])

# creating YearMonth field for the ease of reporting and visualization. 
tx_data['InvoiceYearMonth'] = tx_data['InvoiceDate'].map(lambda date: 100*date.year + date.month)

# calculate revenue for each row and create a new dataframe with yearmonth - Revenue columns
tx_data['Revenue'] = tx_data['UnitPrice'] * tx_data['Quantity']
tx_revenue = tx_data.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()

# tx_revenue.head()

# Need to visualize this

# Next, we can calculate the Monthly Revenue Growth Rate:
# pct_change function captures this well. 
# Make new colun with percent change growth
tx_revenue['MonthlyGrowth'] = tx_revenue['Revenue'].pct_change()

# Now we need to create Monthly Active Customers in order to understand which months had the most customers.

# create a new dataframe with UK customers only:
tx_uk = tx_data.query("Country=='United Kingdom'").reset_index(drop=True)


# need to visualize this

# Monthly Active Customers
tx_monthly_active = tx_uk.groupby('InvoiceYearMonth')['CustomerID'].nunique().reset_index()

# Monthly Order Count
tx_monthly_sales = tx_uk.groupby('InvoiceYearMonth')['Quantity'].sum().reset_index()

# Average Revenue Per Order
tx_monthly_order_avg = tx_uk.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

# New Customer Ratio - Identify new customers by looking at who is new in each segment of time and who is not
tx_min_purchase = tx_uk.groupby('CustomerID').InvoiceDate.min().reset_index() # Create a new df with CustomerID and the first purchase date
tx_min_purchase.columns = ['CustomerID', 'MinPurchaseDate'] # rename columns
tx_min_purchase['MinPurchaseYearMonth'] = tx_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

# Merge first purchase date column to our main dataframe (tx_uk)
tx_uk = pd.merge(tx_uk, tx_min_purchase, on='CustomerID')
# Create a column called User Type and assign 'Existing'
# If Users's First Purchase Year Month before the selected Invoice Year Month, then 'New'
tx_uk['UserType'] = 'New' # just make them all 'New' then overwrite selected 
tx_uk.loc[tx_uk['InvoiceYearMonth']>tx_uk['MinPurchaseYearMonth'],'UserType'] = 'Existing'

# Now create the ratio
tx_user_ratio = tx_uk.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique()/tx_uk.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique()
tx_user_ratio = tx_user_ratio.reset_index()
tx_user_ratio = tx_user_ratio.dropna()

# Monthly Retention Rate - Retained Customers from Previous Month / Active Customers Total
tx_user_purchase = tx_uk.groupby(['CustomerID', 'InvoiceYearMonth'])['Revenue'].sum().astype(int).reset_index()
# create retention matrix with Crosstab
tx_retention = pd.crosstab(tx_user_purchase['CustomerID'], tx_user_purchase['InvoiceYearMonth']).reset_index()

# create an array of dicts that keeps Retained and Total User Counts for each month
months = tx_retention.columns[2:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['InvoiceYearMonth'] = int(selected_month)
    retention_data['TotalUserCount'] = tx_retention[selected_month].sum()
    retention_data['RetainedUserCount'] = tx_retention[(tx_retention[selected_month]>0) & (tx_retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)

tx_retention = pd.DataFrame(retention_array)
tx_retention['RetentionRate'] = tx_retention['RetainedUserCount']/tx_retention['TotalUserCount']

# Cohort Retention Rate 
tx_retention = pd.crosstab(tx_user_purchase['CustomerID'], tx_user_purchase['InvoiceYearMonth']).reset_index()
# create our retention table again with crosstab() - we need to change the column names for using them in .query() function
new_column_names = [ 'm_' + str(column) for column in tx_retention.columns]
tx_retention.columns = new_column_names

retention_array = []
for i in range(len(months)):
    retention_data = {}
    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i+1:]
    for prev_month in prev_months:
        retention_data[prev_month] = np.nan
        
    total_user_count =  retention_data['TotalUserCount'] = tx_retention['m_' + str(selected_month)].sum()
    retention_data[selected_month] = 1 
    
    query = "{} > 0".format('m_' + str(selected_month))
    

    for next_month in next_months:
        query = query + " and {} > 0".format(str('m_' + str(next_month)))
        retention_data[next_month] = np.round(tx_retention.query(query)['m_' + str(next_month)].sum()/total_user_count,2)
    retention_array.append(retention_data)
    
tx_retention = pd.DataFrame(retention_array)
tx_retention.index = months

Part 2: RFM Clustering